In [ ]:
! pip install pandas
! pip install numpy
! pip install matplotlib
! pip install seaborn
! pip install scikit-learn

# 0. Imports

In [2]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


# 1. Carregar o dataset de treino

In [3]:
df = pd.read_csv('./csv/treino/clientes.csv')

In [ ]:
df.head()

# 2. Pré-processamento dos dados

In [ ]:
label_encoders = {}
for column in ['Sexo', 'Profissão', 'Estado Civil']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Preenchendo valores ausentes
df.fillna({'Data de Início no Último Emprego': pd.to_datetime('2000-01-01')}, inplace=True)
df['Data de Início no Último Emprego'] = pd.to_datetime(df['Data de Início no Último Emprego'])
df['Anos no Emprego'] = (pd.to_datetime('today') - df['Data de Início no Último Emprego']).dt.days // 365
df

# 3. Adicionar idade e risco baseado em condições

In [ ]:
df['Data de Nascimento'] = pd.to_datetime(df['Data de Nascimento'])
df['Idade'] = (datetime.now() - df['Data de Nascimento']).dt.days // 365

# Definindo o risco com base na idade e outras condições
def calculate_risk(row):
    risk = 0
    # Condições de risco baseadas na idade
    if 18 <= row['Idade'] <= 24:
        risk += 1  # Risco médio
    elif 25 <= row['Idade'] <= 49:
        risk -= 1  # Risco baixo
    else:
        risk += 1  # Risco médio

    # Adicionando risco se o campo "Data de Início no Último Emprego" for nulo
    if row['Data de Início no Último Emprego'] == pd.to_datetime('2000-01-01'):
        risk += 4  # Aumenta o risco em 80%

    # Adicionando risco baseado no sexo
    if row['Sexo'] == 1:  # Assume que 1 é masculino
        risk += 0.05  # Aumenta 5% para homens

    return risk

df['Risco'] = df.apply(calculate_risk, axis=1)

# Somar o risco ao score de crédito
df['Score Ajustado'] = 500 + (df['Risco'] * 50)  # Inicializa com um score base de 500
df['Score Ajustado'] = df['Score Ajustado'].clip(lower=0, upper=850)

df

# 4. Transformando o score ajustado em classes

In [ ]:
def categorize_score(score):
    if score >= 600:
        return 1  # Bom Pagador
    elif 400 <= score < 600:
        return 0  # Pagador Médio
    else:
        return -1  # Mal Pagador

# Aplicando a categorização
df['Classificacao'] = df['Score Ajustado'].apply(categorize_score)

# Separar em variáveis de entrada e variável alvo
X = df.drop(columns=['Score Ajustado', 'Classificacao', 'Nome', 'Data de Nascimento', 'Data de Início no Último Emprego'])
y = df['Classificacao']

df

In [10]:
# Normalizando os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 5. Treinamento e Teste com KNN

In [11]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

# 6. Treinamento e Teste com Árvore de Decisão

In [12]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

In [ ]:
# 7. Comparando os modelos
print("Resultados do KNN:")
print(classification_report(y_test, y_pred_knn))

print("Resultados da Árvore de Decisão:")
print(classification_report(y_test, y_pred_tree))


# Visualizações (Boxplot, K-Means e DBSCAN)

# 8. Boxplot para detectar outliers

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[['Salário Atual', 'Anos no Emprego']])
plt.title("Detecção de Outliers - Salário Atual e Anos no Emprego")
plt.show()

# 9. K-Means para visualizar clusters de clientes

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Clustering com K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_pca)

plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis')
plt.title("Clusters de Clientes - K-Means")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.show()

# 10. DBSCAN para visualização de quadrantes

In [ ]:

dbscan = DBSCAN(eps=0.5, min_samples=5)
db_clusters = dbscan.fit_predict(X_pca)

plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=db_clusters, cmap='plasma')
plt.title("DBSCAN - Clusters em Quadrantes")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.show()

# Testar um novo dataset

In [18]:

df_teste = pd.read_csv('./csv/teste/clientes_testes.csv')

In [ ]:
df_teste.head()

# Pré-processamento do novo dataset

In [20]:
df_teste['Data de Nascimento'] = pd.to_datetime(df_teste['Data de Nascimento'])
df_teste['Data de Início no Último Emprego'] = pd.to_datetime(df_teste['Data de Início no Último Emprego'], errors='coerce')
df_teste.fillna({'Data de Início no Último Emprego': pd.to_datetime('2000-01-01')}, inplace=True)
df_teste['Idade'] = (datetime.now() - df_teste['Data de Nascimento']).dt.days // 365
df_teste['Anos no Emprego'] = (pd.to_datetime('today') - df_teste['Data de Início no Último Emprego']).dt.days // 365
df_teste['Risco'] = df_teste.apply(calculate_risk, axis=1)
df_teste['Score Ajustado'] = df_teste['Score de Crédito'] + (df_teste['Risco'] * 50)
df_teste['Score Ajustado'] = df_teste['Score Ajustado'].clip(lower=0, upper=850)
df_teste['Classificacao'] = df_teste['Score Ajustado'].apply(categorize_score)


In [21]:
# Preprocessando os dados do novo dataset
X_teste = df_teste[['Sexo', 'Profissão', 'Salário Atual', 'Estado Civil', 'Número de Filhos', 'Empregado', 'Idade', 'Anos no Emprego']]
X_teste = pd.get_dummies(X_teste, drop_first=True)  # One-hot encoding para variáveis categóricas

In [22]:
# Garantindo a mesma ordem das colunas do dataset de treino
X_teste = X_teste.reindex(columns=X.columns, fill_value=0)  # Preencher com 0 para colunas que não estão no teste

X_teste_scaled = scaler.transform(X_teste)  # Normalizando os dados do novo dataset


In [23]:
# Realizando previsões no novo dataset
predicoes_knn = knn.predict(X_teste_scaled)
predicoes_tree = tree.predict(X_teste_scaled)


In [24]:
# Adicionando as previsões ao DataFrame
df_teste['Predicao KNN'] = predicoes_knn
df_teste['Predicao Arvore'] = predicoes_tree

In [ ]:
df_teste

In [27]:
# Função para mapear as previsões para labels descritivos
def map_prediction(pred):
    if pred == 1:
        return "Bom Pagador"
    elif pred == 0:
        return "Pagador Médio"
    else:
        return "Mal Pagador"

In [ ]:
# Imprimindo os resultados das previsões com labels descritivos
df_teste['Predicao KNN'] = df_teste['Predicao KNN'].apply(map_prediction)
df_teste['Predicao Arvore'] = df_teste['Predicao Arvore'].apply(map_prediction)

print("\nResultados das previsões:")
print(df_teste[['Nome', 'Predicao KNN', 'Predicao Arvore']])

# Exportando os resultados para um arquivo CSV

In [29]:
df_teste[['Nome', 'Sexo', 'Profissão', 'Salário Atual', 'Estado Civil', 'Predicao KNN', 'Predicao Arvore']].to_csv('resultados_previsao.csv', index=False)